In [1]:
from src.pandas_pattern_generator import PandasPatternGenerator
from src.lsh import LSHashMap, hamming_dist, norm_vectors
from src.bloom_count import bloom
from src.pattern_overlap import PatternOverlap

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dask.dataframe.hyperloglog import compute_hll_array
import itertools


In [2]:
BITS = 8
NPats = 8

ppg = PandasPatternGenerator(10000, 10)

overlaps = np.asarray([ppg.get_overlap(x, y) for x,y in itertools.product(*[range(1, NPats + 1)]*2)])
overlaps = overlaps.reshape(NPats, NPats).T

In [3]:
overlaps

array([[10000,  5000,  3334,  2500,  2000,  1667,  1429,  1250],
       [ 5000,  5000,  1667,  2500,  1000,  1667,   715,  1250],
       [ 3334,  1667,  3334,   834,   667,  1667,   477,   417],
       [ 2500,  2500,   834,  2500,   500,   834,   358,  1250],
       [ 2000,  1000,   667,   500,  2000,   334,   286,   250],
       [ 1667,  1667,  1667,   834,   334,  1667,   239,   417],
       [ 1429,   715,   477,   358,   286,   239,  1429,   179],
       [ 1250,  1250,   417,  1250,   250,   417,   179,  1250]])

In [ ]:
patterns = [ppg.generate_pattern(x) for x in range(1, NPats + 1)]

# sameLength = True
# if sameLength:
#     N = 10**4

#     pds= [
#         pd.Series(np.arange(N*x)) for x in [1, 2,  4,  10, 20]
#     ] + [pd.Series([1]), pd.Series([1])]
# #     patterns = np.asarray([embed(x, BITS) for x in pds])
#     patterns= np.asarray(pds)

po = PatternOverlap(patterns, LSHwidth=32)
overlaps_close, neighbor_sets = po.get_overlaps(max_ham_distance=10)


print('neighbor sets = \n', neighbor_sets)
print('overlaps = \n', overlaps_close)

> /Users/dimitrikrattiger/Code/hackathon0821/src/pattern_overlap.py(58)embeds()
     56 
     57         breakpoint()
---> 58         return concat_embeds, np.asarray(hll_embeds)
     59 
     60     def get_overlaps(self, max_ham_distance: int = 0):

ipdb> tmp = np.asarray(hll_embeds)
ipdb> type(tmp)
<class 'numpy.ndarray'>
ipdb> tmp.shape
(8, 256)
ipdb> hll_embeds
[array([ 7,  8,  9,  9,  6,  6,  9,  6,  9,  5,  7,  6,  6,  6,  8,  7,  5,
        5,  7, 13,  6,  6,  7,  5,  6,  5,  8,  7,  9, 11,  5,  7,  7,  7,
        7,  7,  8,  4,  7,  6,  6,  6,  6,  6,  6,  6,  8,  9,  5,  6,  5,
        6,  6,  7,  5,  8,  4,  6, 10,  8,  8,  6,  5,  6,  8,  7,  5,  4,
        7,  4,  5,  6,  6,  8,  8,  6,  6,  6,  5,  7,  6, 11,  5,  8,  8,
        8,  7,  6,  5,  4,  8,  7,  4,  7,  5,  6,  8,  6,  5, 12,  6,  6,
        4,  8,  6,  5,  8,  9,  5,  4,  6,  6,  5,  8,  7,  5,  7,  8,  7,
        5,  5,  6,  8,  5,  8,  5,  7,  5,  6, 10,  6,  5,  5,  5,  7, 10,
        4, 10,  5,  7,  5, 11,

In [ ]:
h_dists = np.asarray([hamming_dist(x, y) for x,y in itertools.product(po.lsh.keys(), repeat=2)])
h_dists = h_dists.reshape(len(po.lsh.bins), len(po.lsh.bins))
h_dists

In [ ]:

sinproj = np.zeros((len(patterns), len(patterns)))
sinproj2 = np.zeros((len(patterns), len(patterns)))
for i in range(len(patterns)):
    for j in range(len(patterns)):
        dp = np.dot(po.embs[i,:], po.embs[j,:])
        sinproj[i,j] = dp/(np.linalg.norm(po.embs[i,:]) * np.linalg.norm(po.embs[j,:]))
        sinproj2[i,j] = dp

print(sinproj)
print()
print(sinproj2)

In [ ]:
po.embs.shape

---